# 聚合和分组操作

对大规模数据集进行分析时，一项基本的工作就是进行描述性统计分析，具体来说比如求和、求均值、求中位数、求最大、最小值、求方差等。这些指标呈现了大数据集的总体统计特征。

In [1]:
import numpy as np
import pandas as pd

## 1. 行星数据及简单的统计  
我们通过Seaborn库自带的行星数据来演示。行星数据集包含了天文学家观测到的围绕恒星运转的行星数据。

In [4]:
import seaborn as sns
sns.get_dataset_names()

In D:\Program Files\Miniconda\envs\project2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Program Files\Miniconda\envs\project2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Program Files\Miniconda\envs\project2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Program Files\Miniconda\envs\project2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Program Files\Minicond

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'exercise',
 'flights',
 'fmri',
 'gammas',
 'geyser',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'taxis',
 'tips',
 'titanic']

In [9]:
planets = sns.load_dataset('planets')

这个数据集中包括2014年前已经被发现的一千多颗外行星的信息。

In [10]:
planets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB


可以使用 describe 方法，计算每一列的常用统计值

In [11]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


从以上的统计指标可以看到，1989年首次发现外行星，而且一半的已知外行星都是在2010年及之后的年份发现的。这主要得益于开普特计划，一个通过激光望远镜发现恒星周围椭圆轨道行星的太空计划。

Pandas 内置的一些统计方法：

| 统计方法                  | 描述                            |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

DataFrame 和 Series 支持以上所有方法

## 2. 分组：分割、应用和组合

简单的统计方法仅能针对列的所有数据，我们经常还需要对部分数据进行统计分析，这时就用到了groupby方法了，也就是分组，它包括了三种具体的操作，分别是分割（split），应用（apply）和组合（combine）。

### 2.1 一个经典的“ 分割（Split）, 应用（apply）,组合（ combine）”操作示例
<img src="resource/groupby.jpg" style="zoom:90%">  
<center>groupby 操作的可视化过程</center>  

>*分割步骤将 DataFrame 按照指定的键分割成若干组。  
*应用步骤对每一个组应用函数，通常是累计、转换或过滤函数  
*组合步骤将每一组的结果合并成一个输出的数组

In [12]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)})
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [13]:
df.groupby('key')

值得注意的是，groupby的返回值不是一个 DataFrame 对象，而是一个 DataFrameGroupBy 对象。你可以将它看作是一种特殊形式的 DataFrame，里面包含了几组数据，但是在没有应用函数之前是不会显示出来的。

In [14]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


### 2.2 GroupBy 对象

#### 2.2.1 按列索引

In [15]:
planets.groupby('method') # 对观测方式进行分组

In [16]:
planets.groupby('method')['orbital_period']

我们从原来的 DataFrame 中取某个列名作为一个 Series 数据组，与 GroupBy 对象一样，SeriesGroupBy 对象也只有应用函数，才会显示出数据及结果。  

天体测量学  
日食时间变化  
微透镜  
轨道亮度调制  


In [17]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

这样就可以观察到不同方式下，行星公转周期的中位数

#### 2.2.2 按组迭代

GroupBy 对象支持直接按组进行迭代，返回的每一组都是 Series 或者是 DataFrame。

In [23]:
for method, group in planets.groupby('method'):
    print("{:<30}\t shape={}".format(method, group.shape))

Astrometry                    	 shape=(2, 6)
Eclipse Timing Variations     	 shape=(9, 6)
Imaging                       	 shape=(38, 6)
Microlensing                  	 shape=(23, 6)
Orbital Brightness Modulation 	 shape=(3, 6)
Pulsar Timing                 	 shape=(5, 6)
Pulsation Timing Variations   	 shape=(1, 6)
Radial Velocity               	 shape=(553, 6)
Transit                       	 shape=(397, 6)
Transit Timing Variations     	 shape=(4, 6)


#### 2.2.3 调用方法

In [24]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


这张表可以让我们对数据有更深刻的认识。例如大多数行星都是通过Radial Velocity和Transit方法发现的。而 Transit Timing Variation 和 Orbital Brightness Modulation 方法在2011年后才有发现。